In [1]:
import pandas as pd

from bs4 import BeautifulSoup

from nltk import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer 

In [2]:
from gensim import models
import numpy as np

In [3]:
import gensim
from gensim import corpora
from pprint import pprint
import string

In [4]:
df = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv")

C:\Users\alecr\Anaconda3\envs\gensim_env\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (20,23,28,29,30,31,32,33) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
df.drop('Unnamed: 0',axis=1, inplace=True)

In [6]:
df.head() 

,id,politician_id,riding_id,party_id,end_date,start_date,id.1,document_id,time,h1_en,...,slug,urlcache,h1_fr,h2_fr,h3_fr,who_fr,who_context_fr,wordcount_en,tokenized_content,slug_length
0,2611,3465,35066,4,2005-11-29,1997-09-22,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
1,3210,3465,70224,4,1997-04-27,1994-01-17,232373,1878,2001-05-03 13:50:00-04,Government Orders,...,marlene-catterall-1,/debates/2001/5/3/marlene-catterall-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'on', 'a'...",3
2,4305,173,70358,1,\N,2015-10-19,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
3,1534,173,47007,1,2015-10-19,2004-10-04,645329,388,2008-02-14 13:15:00-05,Routine Proceedings,...,tom-lukiwski-7,/debates/2008/2/14/tom-lukiwski-7/,NaN,NaN,NaN,NaN,NaN,\N,"['finally', ',', 'mr.', 'speaker', ',', 'i', '...",3
4,1541,43,47014,1,2015-10-19,2004-10-04,313253,1621,2004-02-06 10:05:00-05,NaN,...,garry-breitkreuz-1,/debates/2004/2/6/garry-breitkreuz-1/,NaN,NaN,NaN,NaN,NaN,\N,"['mr.', 'speaker', ',', 'i', 'rise', 'today', ...",3


In [ ]:
stop_words = set(stopwords.words('english'))

In [11]:
punctuation = '!"#$%&()*+,-./:;<=>@[]^_`{|}~' + "''"

In [12]:
pun_trans = str.maketrans(dict.fromkeys(punctuation, ' '))

In [18]:
ps = PorterStemmer() 

In [ ]:
%%time
df_chunk = pd.read_csv(r"D:\data\openparliament\statements_nospeaker_en.csv", chunksize=1000)

In [ ]:
content_en_list = []  # append the content from each chunk df here 

In [ ]:
def tokenize_column (row):
    soup = BeautifulSoup(row)
    text = soup.text.lower()
    text = text.translate(pun_trans)
    text = text.replace('\\n', ' ')
    text = text.replace('—', ' ')
    words = word_tokenize(text)
    return words

In [17]:
stop_words = set(stopwords.words('english'))

In [ ]:
def remove_stopwords(word_tokens):
    tokens = [w for w in word_tokens if not w in stop_words]
    tokens = [t for t in tokens if not t.isdigit()]
    words = [ps.stem(w) for w in tokens]
    return words

In [14]:
small_df = df[:10000]

In [19]:
df.columns

Index(['id', 'politician_id', 'riding_id', 'party_id', 'end_date',
       'start_date', 'id.1', 'document_id', 'time', 'h1_en', 'h2_en',
       'member_id', 'who_en', 'content_en', 'sequence_en', 'wordcount',
       'politician_id.1', 'procedural', 'h3_en', 'who_hocid', 'content_fr',
       'statement_type', 'written_question', 'source_id', 'who_context_en',
       'slug', 'urlcache', 'h1_fr', 'h2_fr', 'h3_fr', 'who_fr',
       'who_context_fr', 'wordcount_en', 'tokenized_content', 'slug_length'],
      dtype='object')

In [23]:
%%time
max_tokens = 0
count = 0

for index, row in df.iterrows(): 
    if count%10000==0:
        print(count)
    soup = BeautifulSoup(row.content_en)
    text = soup.text.lower()
    text = text.translate(pun_trans)
    text = text.replace('\\n', ' ')
    text = text.replace('—', ' ')
    word_tokens = word_tokenize(text)
    tokens = [w for w in word_tokens if not w in stop_words]
    tokens = [t for t in tokens if not t.isdigit()]
    words = [ps.stem(w) for w in tokens]
    if len(words)>max_tokens:
        max_tokens = len(words)
    count+=1
print(row.who_en, "gave the longest speech at", max_tokens, 'words')
    
        

0
10000


KeyboardInterrupt: 

In [ ]:
dff = content_en_list[0]

In [ ]:
dff.head()

In [ ]:
len(content_en_list)

In [ ]:
%%time
df_concat = pd.concat(content_en_list)

In [ ]:
df_concat.head()

In [ ]:
politician_id = list(set(df_concat.politician_id))

In [ ]:
df_concat.columns

In [ ]:
d = dict()

In [ ]:
d[4] = 4

In [ ]:
df_small = df_concat[:10]

In [ ]:
df_small.head()

In [ ]:
politicians = list(set(df_concat.politician_id))

In [ ]:
len(politicians)

In [ ]:
vocab_count_dict = dict()

In [ ]:
vocab_count_dict = vocab_count_dict.fromkeys(politicians, set())

In [ ]:
%%time
counter = 0
for index, row in df_concat.iterrows(): 
    if counter%100000 ==0:
        print('at row', counter)
        
    pid = row['politician_id']
    vocab_count_dict[pid].update(row['tokens'])
    counter+=1

In [ ]:
politicians[56]

In [ ]:
vocab_count_dict[4156]

note: filter out '\\n'

In [ ]:
s = 'court.\\nthen'

In [ ]:
'\\n' in s

In [ ]:
s.split('\\n')

In [ ]:
len(vocab_count_dict[4156])

In [ ]:
p_set = vocab_count_dict[4156]

In [ ]:
type(p_set)

In [ ]:
p_set

In [ ]:
p_set.remove('sacrifice.\\ntoday')

In [ ]:
p_set